In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
import os,time
import glob,sys

from astropy.io import fits
from astropy.visualization import ZScaleInterval

from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *

In [2]:
def vmin(data):
    return ZScaleInterval().get_limits(data)[0]
def vmax(data):
    return ZScaleInterval().get_limits(data)[1]

In [ ]:
# Step 0: Get Files from Path

# Observations Ids for SMC_# Survey [50 total]
smc_observation_id = np.arange(40415,40465,1).astype(str)

# Observations Ids for LMC_Survey_# Survey [162 total]
lmc_observation_id = np.arange(45422,45587,1).astype(str)

# Pick which galaxy to do 
galaxies=['LMC','SMC']
gal = galaxies[0]                       # could automate this. 

# Pick an observation ID
observation_ids = lmc_observation_id[49]
print(observation_ids)

for observation_id in observation_ids:
    
    print('Attempting ObsID: ', observation_id)
    t1 = time.time()
    
    # If this obsid has not been run before make it it's own folder
    if not os.path.exists(f"Results/Photometry/{observation_id}"):
        os.mkdir(f"Results/Photometry/{observation_id}")   
    
    def get_hdu_cat_from_file(path,observation_id):
        # TO DO: 
        # Need to update this function so it does all segments
        
        file = f'sw000*{observation_id}*.new'
        filenames = glob.glob(os.path.join(f'Data/UV/{gal}',file))
        print(f"%s files found for id {observation_id}" % len(filenames))
        if len(filenames) == 3: 
            hdus = [fits.open(file)[0] for file in filenames]
            names = [file.split('/')[-1].split('.')[0] for file in filenames]
            catfiles = [f"Data/UV/{gal}/{name}.full.dat" for name in names]
        
            return hdus,catfiles
    
        print(f"Warning: Not format structure that you thought {observation_id}") 
        return [1],[1]
       
    hdu,catname= get_hdu_cat_from_file(path,observation_id) 

    if len(hdu) < 3: continue
    
    meta = {}; bkgd = {}; TractorObjects = {}; 
    Magnitudes = {}; by_filter = {}


    for i,h in enumerate(hdu):
        uvfilter = h.header['FILTER']
        shape = np.shape(h.data)
    
        # Step 1: Get Zaritsky Coordinates and Catalog 
        tz = time.time()
    
        print('---Getting Catalog---')

        catalog = catname[i]
        meta[uvfilter] = get_meta().with_hdu(h,
                                             optical_catalog=catalog,
                                             xdim=[0,shape[1]],
                                             ydim=[0,shape[0]],
                                             cutoff=20.5,
                                             fits_origin=0,
                                             astrometry=True)
    
        print("Time To Run Retrieve Source: ",(time.time()-tz)/60)
    

    
        # Step 2: Error Estimation
        bkgd[uvfilter] = BkgdEstimator(meta[uvfilter],n_pix = [20,20])
    

        # Step 3: Calculate PSF
        pix_scale = np.abs(meta[uvfilter].cdelt)*3600.
        psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                             uvfilter = uvfilter, 
                             width = 23).psf
    
    
        # Step 4: Run Tractor 
        t2 = time.time()
        print('---Starting to Run Tractor---')
        TractorObjects[uvfilter] = PhotometryTools(meta[uvfilter],
                            psf_filename = psf_object,
                            fits_table_sname = f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits", 
                            background = bkgd[uvfilter],
                            fit_positions = False,
                            threshold = 1.5)
        print("Time To Run Tractor: ",(time.time()-t2)/60)
    
        # Step 5: Run Heasarc 
        t3 = time.time()
        Magnitudes[uvfilter] = HeasarcRoutines(f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits",uvfilter)
        print("Time to Run Heasarc: ",(time.time()-t3)/60)
    
        # Step 6: Format CSV
        t4 = time.time()    
    
        df = pd.read_csv(f'{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.csv')

        labels = [uvfilter + '_'+ key for key in df.keys()]
        d = {}
        for old,new in zip(df.keys(),labels):
            d[old] = new
        df = df.rename(columns=d)

        print(f'Joining Optical catalog with {uvfilter}')
        by_filter[uvfilter] = pd.merge(meta[uvfilter].catalog,df,left_on='KEY',right_on=f'{uvfilter}_KEY',how="inner")
        print("Time to Create CSV by Filter: ",(time.time()-t4)/60)
    


    
    # Step 7: Additional Catalog Corrections    
    
    # TO DO: Check what filters are present, we want all info regardless if all filters are present. 
    
    #Merge Each Filter into One Catalog 
    
    #Try only keeping things where errors are good.
    by_filter['UVM2'] = by_filter['UVM2'][by_filter['UVM2'].UVM2_MAG_ERR < 0.35]
    by_filter['UVW1'] = by_filter['UVW1'][by_filter['UVW1'].UVW1_MAG_ERR < 0.35]
    by_filter['UVW2'] = by_filter['UVW2'][by_filter['UVW2'].UVW2_MAG_ERR < 0.35]

    #Orignial Stuff:
    first_two = pd.merge(by_filter['UVM2'],by_filter['UVW2'],on=['Ra','Dec'],how='outer')
    catalog = pd.merge(first_two,by_filter['UVW1'],on=['Ra','Dec'],how='outer')

    #  Remove duplicate optical photometry
    catalog = catalog.drop(labels=['Umag_y', 'e_Umag_y', 'Bmag_y', 'e_Bmag_y', 'Vmag_y',
                                   'e_Vmag_y', 'Imag_y', 'e_Imag_y', 'Flag_y', 'Jmag_y', 
                                   'e_Jmag_y','Hmag_y', 'e_Hmag_y', 'Ksmag_y', 'e_Ksmag_y',
                                   'Umag_x', 'e_Umag_x', 'Bmag_x', 'e_Bmag_x', 'Vmag_x',
                                   'e_Vmag_x', 'Imag_x', 'e_Imag_x', 'Flag_x', 'Jmag_x', 
                                   'e_Jmag_x','Hmag_x', 'e_Hmag_x', 'Ksmag_x', 'e_Ksmag_x'],
                          axis=1)

    # Might as well not keep things with no photometry 
    catalog = catalog[(np.isfinite(catalog.UVM2_RA)) | (np.isfinite(catalog.UVW1_RA)) | (np.isfinite(catalog.UVW2_RA)) ]
    catalog.to_csv(f"{path}/{observation_id}/{observation_id}_1_1.csv")

    # Step 8 Plot Tractor Results 

    f,axes=plt.subplots(3,3,figsize=(20,20))
    fontsize = 20
    for i,h in enumerate(hdu):
        uvfilter = h.header['FILTER']
    
        data = TractorObjects[uvfilter].image
        model = TractorObjects[uvfilter].model
        residual = data - model 

        axes[i,0].imshow(data,vmin=vmin(data),vmax=vmax(data),origin="lower")
        axes[i,1].imshow(model,vmin=vmin(data),vmax=vmax(data),origin="lower")
        axes[i,2].imshow(residual,vmin=vmin(residual),vmax=vmax(residual),origin="lower")
    
        axes[i,0].set_ylabel(uvfilter,fontsize=fontsize)


    plt.savefig(f"{path}/{observation_id}/{observation_id}_1_1_tractor.png")
    #plt.close()  


    # Step 9 Plot Photometry 

    #plt.style.use('bmh')
    
    g,axs=plt.subplots(2,4,figsize=(30,20))

    uvm2_v = catalog[(catalog.UVM2_MAG_ERR < 0.35) & (catalog.UVM2_SATURATED == False) & (catalog.UVM2_SSS == 1.0) & (catalog.UVM2_EDGE == 1.0) & (catalog.e_Vmag < 0.35)]
    uvw1_v = catalog[(catalog.UVW1_MAG_ERR < 0.35) & (catalog.UVW1_SATURATED == False) & (catalog.UVW1_SSS == 1.0) & (catalog.UVW1_EDGE == 1.0) & (catalog.e_Vmag < 0.35)]
    uvw2_v = catalog[(catalog.UVW2_MAG_ERR < 0.35) & (catalog.UVW2_SATURATED == False) & (catalog.UVW2_SSS == 1.0) & (catalog.UVW2_EDGE == 1.0) & (catalog.e_Vmag < 0.35)]

    uvm2_uvw1 = catalog[(catalog.UVM2_MAG_ERR < 0.35) & (catalog.UVM2_SATURATED == False) & (catalog.UVM2_SSS == 1.0) & 
                        (catalog.UVM2_EDGE == 1.0) & (catalog.UVW1_MAG_ERR < 0.35) & (catalog.UVW1_SATURATED == False) & 
                        (catalog.UVW1_SSS == 1.0) & (catalog.UVW1_EDGE == 1.0)]

    axs[0,0].scatter(uvw1_v.UVW1_MAG - uvw1_v.Vmag,uvw1_v.UVW1_MAG)
    axs[0,1].scatter(uvm2_v.UVM2_MAG - uvm2_v.Vmag,uvm2_v.UVM2_MAG )
    axs[0,2].scatter(uvw2_v.UVW2_MAG - uvw2_v.Vmag,uvw2_v.UVW2_MAG)
    axs[0,3].scatter(uvm2_uvw1.UVM2_MAG - uvm2_uvw1.UVW1_MAG,uvm2_uvw1.UVM2_MAG)

    axs[1,0].scatter(uvw1_v.UVW1_MAG - uvw1_v.Vmag,uvw1_v.Vmag)
    axs[1,1].scatter(uvm2_v.UVM2_MAG - uvm2_v.Vmag,uvm2_v.Vmag )
    axs[1,2].scatter(uvw2_v.UVW2_MAG - uvw2_v.Vmag,uvw2_v.Vmag)
    axs[1,3].scatter(uvm2_uvw1.UVM2_MAG - uvm2_uvw1.UVW1_MAG,uvm2_uvw1.UVW1_MAG)

    axs[0,0].set_xlabel("UVW1 - V",fontsize=fontsize); axs[0,0].set_ylabel("UVW1",fontsize=fontsize);
    axs[0,1].set_xlabel("UVM2 - V",fontsize=fontsize); axs[0,1].set_ylabel("UVM2",fontsize=fontsize);
    axs[0,2].set_xlabel("UVW2 - V",fontsize=fontsize); axs[0,2].set_ylabel("UVW2",fontsize=fontsize);
    axs[0,3].set_xlabel("UVM2 - UVW1",fontsize=fontsize); axs[0,3].set_ylabel("UVM2",fontsize=fontsize);

    axs[1,0].set_xlabel("UVW1 - V",fontsize=fontsize); axs[1,0].set_ylabel("V",fontsize=fontsize);
    axs[1,1].set_xlabel("UVM2 - V",fontsize=fontsize); axs[1,1].set_ylabel("V",fontsize=fontsize);
    axs[1,2].set_xlabel("UVW2 - V",fontsize=fontsize); axs[1,2].set_ylabel("V",fontsize=fontsize);
    axs[1,3].set_xlabel("UVM2 - UVW1",fontsize=fontsize); axs[1,3].set_ylabel("UVW1",fontsize=fontsize);

    [ax.set_ylim(19,11) for ax in axs[0]]
    [ax.set_xlim(-7,4) for ax in axs[0]]

    [ax.set_ylim(20,12) for ax in axs[1]]
    [ax.set_xlim(-7,4) for ax in axs[1]]


    plt.savefig(f"{path}/{observation_id}/{observation_id}_1_1_photometry.png")
    #plt.close()  

    # Step 10 Plot Coordinates 
    plt.figure(figsize=(10,10))

    d = meta['UVM2'].data
    x,y = meta['UVM2'].pixel_positions

    plt.imshow(d,vmin=vmin(d),vmax=vmax(d),origin="lower")
    plt.scatter(x,y,s=5,c='red')

    plt.xlim(500,600)
    plt.ylim(500,600)

    plt.savefig(f"{path}/{observation_id}/{observation_id}_1_1coordinates.png")

    print("Total Time for ObsID: ",observation_id,' (1_1) ', (time.time()-t1)/60)        

['45471' '45472' '45473']
Attempting ObsID:  45471
3 files found for id 45471
---Getting Catalog---


INFO:astropy:
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
Time To Run Retrieve

INFO:astropy:
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
